<a href="https://colab.research.google.com/github/Komal77rao/Data-Eng-Modules/blob/main/4-request-api-object/index-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Client Class

### Introduction

Ok, so far we have created two different classes, a `Receipt` class and a `ReceiptBuilder` class.  Let's review.

1. Our Receipt class
    * This class simply contains setters and getters for the attributes that we find important.

In [ ]:
class Receipt:
    def __init__(self, total, address, end_date, restaurant_name):
        self.total = int(total)
        self.address = address
        self.end_date = end_date
        self.restaurant_name = restaurant_name

2. The ReceiptBuilder class constructs new objects.

In [ ]:
import requests
class ReceiptBuilder:
    def run(self):
        self._receipts_data = self.retrieve_receipts()
        self._receipts = self.receipts_data_to_objects(self._receipts_data)
        return self._receipts

    def retrieve_receipts(self):
        response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
        return response.json()

    def create_receipt(self, receipt_data):
        receipt = Receipt(receipt_data['total_receipts'], receipt_data['location_address'],
                          receipt_data['obligation_end_date_yyyymmdd'], receipt_data['location_name'])
        return receipt

    def receipts_data_to_objects(self, receipts_data):
        receipts = []
        for receipt_data in receipts_data:
            receipt = self.create_receipt(receipt_data)
            receipts.append(receipt)
        return receipts

### Going further

Now looking at the code, there is some code that is different than the other.  We can identify it by looking at our `run` method.

```python
def run(self):
    self._receipts_data = self.retrieve_receipts()
    self._receipts = self.receipts_data_to_objects(self._receipts_data)
    return self._receipts
```

We can see that this method does two things:
1. retrieve data from the api
2. coerce the data into Receipt objects.  

These are two tasks that we may not always want to perform together.  For example, we may want to retreive data from the API and create new restaurant objects, or new location objects.  So it makes sense to isolate this code.  Here's how.

1. We just move the `retrieve_receipts` method into a new class.

In [1]:
class TexasDrinksClient:
    URL = "https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE"
    def run(self):
        response = requests.get(self.URL)
        return response.json()

And then we remove the related code from the `ReceiptBuilder`, and replace it with a call to the `TexasDrinksClient`.

In [2]:
import requests
class ReceiptBuilder:
    def run(self):
      #add the TexasDrinksClient class here
        self._request_api = TexasDrinksClient()
        self._receipts_data = self._request_api.run()
        self._receipts = self.receipts_data_to_objects(self._receipts_data)
        return self._receipts

    # remove retrieve_receipts method

    def select_data(self, receipt_data):
        attributes = ('total_receipts', 'location_address', 'obligation_end_date_yyyymmdd', 'location_name')
        return dict((k, receipt_data[k]) for k in attributes)

    def create_receipt(self, receipt_data):
        receipt = Receipt(receipt_data['total_receipts'], receipt_data['location_address'], receipt_data['obligation_end_date_yyyymmdd'], receipt_data['location_name'])
        return receipt

    def receipts_data_to_objects(self, receipts_data):
        receipts = []
        for receipt_data in receipts_data:
            selected_data = self.select_data(receipt_data)
            receipt = self.create_receipt(selected_data)
            receipts.append(receipt)
        return receipts

And then we check to make sure that this still works.

In [3]:
builder = ReceiptBuilder()
receipts = builder.run()
receipts[0:3]

NameError: ignored

### Such small improvements?

It may seem silly to create a class for just one method.  Why do we?

In [4]:
class TexasDrinksAPI:
    URL = "https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE"
    def run(self):
        response = requests.get(self.URL)
        return response.json()

In [7]:
client = TexasDrinksAPI()

client.run()[0]

{'taxpayer_number': '32046798537',
 'taxpayer_name': 'MWD DALLAS UPTOWN, LLC',
 'taxpayer_address': '1000 N POST OAK RD STE 220',
 'taxpayer_city': 'HOUSTON',
 'taxpayer_state': 'TX',
 'taxpayer_zip': '77055',
 'taxpayer_county': '101',
 'location_number': '1',
 'location_name': "MAX'S WINE DIVE",
 'location_address': '3600 MCKINNEY AVE STE 100',
 'location_city': 'DALLAS',
 'location_state': 'TX',
 'location_zip': '75204',
 'location_county': '57',
 'inside_outside_city_limits_code_y_n': 'Y',
 'tabc_permit_number': 'MB917035',
 'responsibility_begin_date_yyyymmdd': '2015-08-11T00:00:00.000',
 'responsibility_end_date_yyyymmdd': '2017-08-21T00:00:00.000',
 'obligation_end_date_yyyymmdd': '2015-08-31T00:00:00.000',
 'liquor_receipts': '0',
 'wine_receipts': '0',
 'beer_receipts': '0',
 'cover_charge_receipts': '0',
 'total_receipts': '0'}

In part we do this because we are following a standard recipe.  Generally, when we make a request to an API we require multiple pieces of information like the root_url, the client_id and the client_secret to make a request.

Even above we could further refactor our Client class.

In [8]:
class TexasDrinksAPI:
    ROOT_URL = "https://data.texas.gov/resource/naix-2893.json"

    def run(self, params = {'location_name': 'HYATT HOUSE DALLAS/UPTOWN'}):
        response = requests.get(self.ROOT_URL, params)
        return response.json()

In [9]:
client = TexasDrinksAPI()

client.run()[0].keys()

dict_keys(['taxpayer_number', 'taxpayer_name', 'taxpayer_address', 'taxpayer_city', 'taxpayer_state', 'taxpayer_zip', 'taxpayer_county', 'location_number', 'location_name', 'location_address', 'location_city', 'location_state', 'location_zip', 'location_county', 'inside_outside_city_limits_code_y_n', 'tabc_permit_number', 'responsibility_begin_date_yyyymmdd', 'obligation_end_date_yyyymmdd', 'liquor_receipts', 'wine_receipts', 'beer_receipts', 'cover_charge_receipts', 'total_receipts'])

### Summary

In this lesson, we learned to remove code related to the API into it's own class.  The reason why we do this is because requesting the Texas API has value even when not related to building receipts.  And we want an easy tool to do this.

We also saw a standard pattern for reaching an API.  The main idea is that we do not like working with messy dictionaries and instead would prefer to work with objects.  We accomplish this by creating three classes:

1. A `TexasDrinksAPI` class that makes requests to the API
2. A `ReceiptBuilder` class that turns the data into `Receipt` instances
3. The `Receipt` class